In [1]:
import pandas as pd

In [2]:
bat_df = pd.read_csv('../Datasets/batter_stats.csv')
pitch_df = pd.read_csv('../Datasets/pitcher_stats.csv')
lineups = pd.read_csv('../Datasets/Lineups_2021_06_11.csv')

In [3]:
# The dots in names like J.T. were inconsistent across datasets
bat_df.loc[:,'Name'] = bat_df['Name'].str.replace('.', '')
pitch_df.loc[:,'Name'] = pitch_df['Name'].str.replace('.', '')
lineups.loc[:,' player name'] = lineups[' player name'].str.replace('.', '')

In [4]:
bats_r = bat_df[bat_df['bats'] == 'R']
bats_l = bat_df[bat_df['bats'] == 'L']

In [5]:
# Making name column consistent
lineups.columns = lineups.columns.str.replace(' player name', 'Name')

In [6]:
# Individual name that was giving trouble
lineups.loc[lineups['Name'] == 'Jonathan Gray', 'Name'] = 'Jon Gray'

In [7]:
# Creates a list of teams with no repeats
# The order needs to be kept from the dataframe because 
# opposing teams are paired
team_list = []
for team in lineups['team code']:
    if team not in team_list:
        team_list.append(team)
# Creating a dictionary of every team, and the team they face
matchups = {}
for i in range(1, len(team_list), 2):
    matchups[team_list[i]] = team_list[i-1]
    matchups[team_list[i-1]] = team_list[i]

In [8]:
lineups

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position
0,STL,6/11/2021,1,669242.0,Tommy Edman,1,Y,RF
1,STL,6/11/2021,1,666185.0,Dylan Carlson,2,Y,CF
2,STL,6/11/2021,1,502671.0,Paul Goldschmidt,3,Y,1B
3,STL,6/11/2021,1,571448.0,Nolan Arenado,4,Y,3B
4,STL,6/11/2021,1,641933.0,Tyler O'Neill,5,Y,LF
...,...,...,...,...,...,...,...,...
259,LAD,6/11/2021,1,669257.0,Will Smith,5,N,NaN
260,LAD,6/11/2021,1,621035.0,Chris Taylor,6,N,NaN
261,LAD,6/11/2021,1,666158.0,Gavin Lux,7,N,NaN
262,LAD,6/11/2021,1,572041.0,AJ Pollock,8,N,NaN


In [9]:
batters = lineups.loc[(lineups[' batting order'] != 'SP') & (lineups[' game_number'] == 1)]

In [10]:
pitch_df.drop(456, inplace = True)

In [11]:
matchups

{'CHC': 'STL',
 'STL': 'CHC',
 'WAS': 'SF',
 'SF': 'WAS',
 'NYM': 'SD',
 'SD': 'NYM',
 'CLE': 'SEA',
 'SEA': 'CLE',
 'CIN': 'COL',
 'COL': 'CIN',
 'BOS': 'TOR',
 'TOR': 'BOS',
 'MIA': 'ATL',
 'ATL': 'MIA',
 'DET': 'CHW',
 'CHW': 'DET',
 'TB': 'BAL',
 'BAL': 'TB',
 'MIN': 'HOU',
 'HOU': 'MIN',
 'MIL': 'PIT',
 'PIT': 'MIL',
 'OAK': 'KC',
 'KC': 'OAK',
 'ARI': 'LAA',
 'LAA': 'ARI',
 'LAD': 'TEX',
 'TEX': 'LAD'}

In [12]:
pitch_df.loc[pitch_df['Name']== 'Chase DeJong', 'throws']

Series([], Name: throws, dtype: object)

In [13]:
opp_pitch_hand = []
opp_pitcher = []
del_indexes = []
keep_pitch = []
for i, row in batters.iterrows():
    opp_pitch = lineups.loc[(lineups['team code'] == matchups[row['team code']])\
                            & (lineups[' batting order'] == 'SP') & (lineups[' game_number'] == 1), 'Name'].values[0]
    if len(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values) > 0:
            opp_pitch_hand.append(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values[0])
            opp_pitcher.append(opp_pitch)
            keep_pitch.append(opp_pitch)
    else:
        del_indexes.append(i)
        
batters.drop(index = del_indexes, inplace = True)
batters['opp_pitch_hand'] = opp_pitch_hand
batters['opp_pitch'] = opp_pitcher


/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-13-9eb6f881f0bf>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batters['opp_pitch_hand'] = opp_pitch_hand
<ipython-input-13-9eb6f881f0bf>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [14]:
del_indexes

[199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244]

In [15]:
bats = []
k_pct = []
bb_pct = []
bb_k = []
bat_avg = []
obp = []
slg = []
ops = []
iso = []
babip = []
wrc = []
wraa = []
woba = []
wrc_plus = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    if row['opp_pitch_hand'] == 'R':
        batter_df = bat_df.loc[(bat_df['pitch']== 'R')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'L']
    if row['opp_pitch_hand'] == 'L':
        batter_df = bat_df.loc[(bat_df['pitch']== 'L')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'R']
    if len(batter_df) == 0:
        for col in batter_df.columns:
            batter_df[col] = [float('NaN')]
    bats.append(batter_df['bats'].values[0])
    k_pct.append(batter_df['K%'].values[0])
    bb_pct.append(batter_df['BB%'].values[0])
    bb_k.append(batter_df['BB/K'].values[0])
    bat_avg.append(batter_df['AVG'].values[0])
    obp.append(batter_df['OBP'].values[0])
    slg.append(batter_df['SLG'].values[0])
    ops.append(batter_df['OPS'].values[0])
    iso.append(batter_df['ISO'].values[0])
    babip.append(batter_df['BABIP'].values[0])
    wrc.append(batter_df['wRC'].values[0])
    wraa.append(batter_df['wRAA'].values[0])
    woba.append(batter_df['wOBA'].values[0])
    wrc_plus.append(batter_df['wRC+'].values[0])
batters.loc[:,'bats'] = bats
batters.loc[:,'k_pct'] = k_pct
batters.loc[:,'bb_pct'] = bb_pct
batters.loc[:,'bb_k'] = bb_k
batters.loc[:,'bat_avg'] = bat_avg
batters.loc[:,'obp'] = obp
batters.loc[:,'slg'] = slg
batters.loc[:,'ops'] = ops
batters.loc[:,'iso'] = iso
batters.loc[:,'babip'] = babip
batters.loc[:,'wrc'] = wrc
batters.loc[:,'wraa'] = wraa
batters.loc[:,'woba'] = woba
batters.loc[:,'wrc_plus'] = wrc_plus

<ipython-input-15-0499cefeeb18>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batter_df[col] = [float('NaN')]
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [16]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,bat_avg,obp,slg,ops,iso,babip,wrc,wraa,woba,wrc_plus
0,STL,6/11/2021,1,669242.0,Tommy Edman,1,Y,RF,R,Kohl Stewart,...,0.252941,0.316489,0.347059,0.663548,0.094118,0.293478,39.487438,-6.013589,0.295345,87.708270
1,STL,6/11/2021,1,666185.0,Dylan Carlson,2,Y,CF,R,Kohl Stewart,...,0.231760,0.309160,0.369099,0.678259,0.137339,0.308176,27.873883,-3.611760,0.297408,89.481313
2,STL,6/11/2021,1,502671.0,Paul Goldschmidt,3,Y,1B,R,Kohl Stewart,...,0.280864,0.358904,0.435185,0.794089,0.154321,0.337500,54.618967,10.212394,0.348034,122.352497
3,STL,6/11/2021,1,571448.0,Nolan Arenado,4,Y,3B,R,Kohl Stewart,...,0.263699,0.319876,0.486301,0.806177,0.222603,0.256098,44.620267,5.781619,0.336688,107.158994
4,STL,6/11/2021,1,641933.0,Tyler O'Neill,5,Y,LF,R,Kohl Stewart,...,0.212871,0.274775,0.480198,0.754973,0.267327,0.223140,27.710981,0.692501,0.320407,103.784961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,LAD,6/11/2021,1,641355.0,Cody Bellinger,4,N,NaN,R,Mike Foltynewicz,...,0.246753,0.344633,0.493506,0.838139,0.246753,0.245763,27.390634,5.323999,0.354311,125.907423
259,LAD,6/11/2021,1,669257.0,Will Smith,5,N,NaN,R,Mike Foltynewicz,...,0.308642,0.425000,0.604938,1.029938,0.296296,0.299213,43.343418,19.158044,0.431390,178.210642
260,LAD,6/11/2021,1,621035.0,Chris Taylor,6,N,NaN,R,Mike Foltynewicz,...,0.283333,0.392226,0.450000,0.842226,0.166667,0.372671,47.437224,12.948708,0.370080,137.743822
261,LAD,6/11/2021,1,666158.0,Gavin Lux,7,N,NaN,R,Mike Foltynewicz,...,0.234177,0.290698,0.398734,0.689432,0.164557,0.271930,18.268869,-2.303424,0.296797,90.348827


In [17]:
k_9 = []
bb_9 = []
k_bb = []
hr_9 = []
k_pct = []
bb_pct = []
k_bb_pct = []
bat_avg = []
whip = []
babip = []
lob_pct = []
fip = []
xfip = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    pitcher_df = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== row['bats'])&(pitch_df['Name']==row['opp_pitch']),:]
    if len(pitcher_df) == 0:
        for col in pitcher_df.columns:
            pitcher_df[col] = [float('NaN')]
    k_9.append(pitcher_df['K/9'].values[0])
    bb_9.append(pitcher_df['BB/9'].values[0])
    k_bb.append(pitcher_df['K/BB'].values[0])
    hr_9.append(pitcher_df['HR/9'].values[0])
    k_pct.append(pitcher_df['K%'].values[0])
    bb_pct.append(pitcher_df['BB%'].values[0])
    k_bb_pct.append(pitcher_df['K-BB%'].values[0])
    bat_avg.append(pitcher_df['AVG'].values[0])
    whip.append(pitcher_df['WHIP'].values[0])
    babip.append(pitcher_df['BABIP'].values[0])
    lob_pct.append(pitcher_df['LOB%'].values[0])
    fip.append(pitcher_df['FIP'].values[0])
    xfip.append(pitcher_df['xFIP'].values[0])
batters.loc[:,'opp_k/9'] = k_9
batters.loc[:,'opp_bb/9'] = k_9
batters.loc[:,'opp_k/bb'] = k_9
batters.loc[:,'opp_hr/9'] = k_9
batters.loc[:,'opp_k_pct'] = k_9
batters.loc[:,'opp_bb_pct'] = k_9
batters.loc[:,'opp_k_bb_pct'] = k_9
batters.loc[:,'opp_avg'] = k_9
batters.loc[:,'opp_whip'] = k_9
batters.loc[:,'opp_babip'] = k_9
batters.loc[:,'opp_lob_pct'] = k_9
batters.loc[:,'opp_fip'] = k_9
batters.loc[:,'opp_xfip'] = k_9

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [18]:
pitchers = lineups.loc[(lineups[' batting order'] == 'SP')]

In [19]:
pitchers = pitchers[pitchers['Name'].isin(keep_pitch)]

In [20]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,opp_hr/9,opp_k_pct,opp_bb_pct,opp_k_bb_pct,opp_avg,opp_whip,opp_babip,opp_lob_pct,opp_fip,opp_xfip
0,STL,6/11/2021,1,669242.0,Tommy Edman,1,Y,RF,R,Kohl Stewart,...,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000
1,STL,6/11/2021,1,666185.0,Dylan Carlson,2,Y,CF,R,Kohl Stewart,...,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000
2,STL,6/11/2021,1,502671.0,Paul Goldschmidt,3,Y,1B,R,Kohl Stewart,...,10.125000,10.125000,10.125000,10.125000,10.125000,10.125000,10.125000,10.125000,10.125000,10.125000
3,STL,6/11/2021,1,571448.0,Nolan Arenado,4,Y,3B,R,Kohl Stewart,...,10.125000,10.125000,10.125000,10.125000,10.125000,10.125000,10.125000,10.125000,10.125000,10.125000
4,STL,6/11/2021,1,641933.0,Tyler O'Neill,5,Y,LF,R,Kohl Stewart,...,10.125000,10.125000,10.125000,10.125000,10.125000,10.125000,10.125000,10.125000,10.125000,10.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,LAD,6/11/2021,1,641355.0,Cody Bellinger,4,N,NaN,R,Mike Foltynewicz,...,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000
259,LAD,6/11/2021,1,669257.0,Will Smith,5,N,NaN,R,Mike Foltynewicz,...,6.967742,6.967742,6.967742,6.967742,6.967742,6.967742,6.967742,6.967742,6.967742,6.967742
260,LAD,6/11/2021,1,621035.0,Chris Taylor,6,N,NaN,R,Mike Foltynewicz,...,6.967742,6.967742,6.967742,6.967742,6.967742,6.967742,6.967742,6.967742,6.967742,6.967742
261,LAD,6/11/2021,1,666158.0,Gavin Lux,7,N,NaN,R,Mike Foltynewicz,...,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000,5.400000


In [21]:
num_L_bats = []
num_R_bats = []
for i, row in pitchers.iterrows():
    opp_lineup = batters.loc[(batters['team code'] == matchups[row['team code']]),:]
    find_batters = []
    # There are batters whos data was null due to lack of plate appearances but handedness is known
    for j, row2 in opp_lineup.iterrows():
        if row2['bats'] != 'R' and row2['bats'] != 'L':
            find_bats = bat_df.loc[bat_df['Name'] == row2['Name'],:]
            if len(find_bats) > 0:
                find_batters.append((j, find_bats['bats'].values[0]))
            # If batter handedness is unknown, right is more likely
            if len(find_bats) == 0:
                find_batters.append((j, 'R'))
    for handedness in find_batters:
        opp_lineup.loc[handedness[0], 'bats'] = handedness[1]
    try:
        num_L_bats.append(opp_lineup['bats'].value_counts()['L'])
    except:
        num_L_bats.append(0)
    try:
        num_R_bats.append(opp_lineup['bats'].value_counts()['R'])
    except:
        num_R_bats.append(0)
pitchers['num_L_bats'] = num_L_bats
pitchers['num_R_bats'] = num_R_bats
is_nl = []
for i, row in pitchers.iterrows():
    if row['num_L_bats'] + row['num_R_bats'] == 9:
        is_nl.append(0)
    else:
        is_nl.append(1)
pitchers['is_nl'] = is_nl

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [22]:
k_9_l = []
bb_9_l = []
k_bb_l = []
hr_9_l = []
k_pct_l = []
bb_pct_l = []
k_bb_pct_l = []
avg_l = []
whip_l = []
babip_l = []
lob_pct_l = []
fip_l = []
xfip_l = []
k_9_r = []
bb_9_r = []
k_bb_r = []
hr_9_r = []
k_pct_r = []
bb_pct_r = []
k_bb_pct_r = []
avg_r = []
whip_r = []
babip_r = []
lob_pct_r = []
fip_r = []
xfip_r = []
# Engineering features
k_9_l_tot = []
bb_9_l_tot = []
k_bb_l_tot = []
hr_9_l_tot = []
k_pct_l_tot = []
bb_pct_l_tot = []
k_bb_pct_l_tot = []
avg_l_tot = []
whip_l_tot = []
babip_l_tot = []
lob_pct_l_tot = []
fip_l_tot = []
xfip_l_tot = []
k_9_r_tot = []
bb_9_r_tot = []
k_bb_r_tot = []
hr_9_r_tot = []
k_pct_r_tot = []
bb_pct_r_tot = []
k_bb_pct_r_tot = []
avg_r_tot = []
whip_r_tot = []
babip_r_tot = []
lob_pct_r_tot = []
fip_r_tot = []
xfip_r_tot = []
for i, row in pitchers.iterrows():
    # Pulling stats of pitcher against lefty hitters
    pitcher_df_l = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'L')&(pitch_df['Name']==row['Name']),:]
    if len(pitcher_df_l) == 0:
        for col in pitcher_df_l.columns:
            pitcher_df_l[col] = [float('NaN')]
    k_9_l.append(pitcher_df_l['K/9'].values[0])
    bb_9_l.append(pitcher_df_l['BB/9'].values[0])
    k_bb_l.append(pitcher_df_l['K/BB'].values[0])
    hr_9_l.append(pitcher_df_l['HR/9'].values[0])
    k_pct_l.append(pitcher_df_l['K%'].values[0])
    bb_pct_l.append(pitcher_df_l['BB%'].values[0])
    k_bb_pct_l.append(pitcher_df_l['K-BB%'].values[0])
    avg_l.append(pitcher_df_l['AVG'].values[0])
    whip_l.append(pitcher_df_l['WHIP'].values[0])
    babip_l.append(pitcher_df_l['BABIP'].values[0])
    lob_pct_l.append(pitcher_df_l['LOB%'].values[0])
    fip_l.append(pitcher_df_l['FIP'].values[0])
    xfip_l.append(pitcher_df_l['xFIP'].values[0])
    # Feature Engineering
    k_9_l_tot.append(pitcher_df_l['K/9'].values[0] * row['num_L_bats'])
    bb_9_l_tot.append(pitcher_df_l['BB/9'].values[0] * row['num_L_bats'])
    k_bb_l_tot.append(pitcher_df_l['K/BB'].values[0] * row['num_L_bats'])
    hr_9_l_tot.append(pitcher_df_l['HR/9'].values[0] * row['num_L_bats'])
    k_pct_l_tot.append(pitcher_df_l['K%'].values[0] * row['num_L_bats'])
    bb_pct_l_tot.append(pitcher_df_l['BB%'].values[0] * row['num_L_bats'])
    k_bb_pct_l_tot.append(pitcher_df_l['K-BB%'].values[0] * row['num_L_bats'])
    avg_l_tot.append(pitcher_df_l['AVG'].values[0] * row['num_L_bats'])
    whip_l_tot.append(pitcher_df_l['WHIP'].values[0] * row['num_L_bats'])
    babip_l_tot.append(pitcher_df_l['BABIP'].values[0] * row['num_L_bats'])
    lob_pct_l_tot.append(pitcher_df_l['LOB%'].values[0] * row['num_L_bats'])
    fip_l_tot.append(pitcher_df_l['FIP'].values[0] * row['num_L_bats'])
    xfip_l_tot.append(pitcher_df_l['xFIP'].values[0] * row['num_L_bats'])
    # Pulling stats against righty hitters
    pitcher_df_r = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'R')&(pitch_df['Name']== row['Name']),:]
    if len(pitcher_df_r) == 0:
        for col in pitcher_df_r.columns:
            pitcher_df_r[col] = [float('NaN')]
    k_9_r.append(pitcher_df_r['K/9'].values[0])
    bb_9_r.append(pitcher_df_r['BB/9'].values[0])
    k_bb_r.append(pitcher_df_r['K/BB'].values[0])
    hr_9_r.append(pitcher_df_r['HR/9'].values[0])
    k_pct_r.append(pitcher_df_r['K%'].values[0])
    bb_pct_r.append(pitcher_df_r['BB%'].values[0])
    k_bb_pct_r.append(pitcher_df_r['K-BB%'].values[0])
    avg_r.append(pitcher_df_r['AVG'].values[0])
    whip_r.append(pitcher_df_r['WHIP'].values[0])
    babip_r.append(pitcher_df_r['BABIP'].values[0])
    lob_pct_r.append(pitcher_df_r['LOB%'].values[0])
    fip_r.append(pitcher_df_r['FIP'].values[0])
    xfip_r.append(pitcher_df_r['xFIP'].values[0])
    # Feature Engineering
    k_9_r_tot.append(pitcher_df_r['K/9'].values[0] * row['num_R_bats'])
    bb_9_r_tot.append(pitcher_df_r['BB/9'].values[0] * row['num_R_bats'])
    k_bb_r_tot.append(pitcher_df_r['K/BB'].values[0] * row['num_R_bats'])
    hr_9_r_tot.append(pitcher_df_r['HR/9'].values[0] * row['num_R_bats'])
    k_pct_r_tot.append(pitcher_df_r['K%'].values[0] * row['num_R_bats'])
    bb_pct_r_tot.append(pitcher_df_r['BB%'].values[0] * row['num_R_bats'])
    k_bb_pct_r_tot.append(pitcher_df_r['K-BB%'].values[0] * row['num_R_bats'])
    avg_r_tot.append(pitcher_df_r['AVG'].values[0] * row['num_R_bats'])
    whip_r_tot.append(pitcher_df_r['WHIP'].values[0] * row['num_R_bats'])
    babip_r_tot.append(pitcher_df_r['BABIP'].values[0] * row['num_R_bats'])
    lob_pct_r_tot.append(pitcher_df_r['LOB%'].values[0] * row['num_R_bats'])
    fip_r_tot.append(pitcher_df_r['FIP'].values[0] * row['num_R_bats'])
    xfip_r_tot.append(pitcher_df_r['xFIP'].values[0] * row['num_R_bats'])
pitchers['k_9_l'] = k_9_l
pitchers['bb_9_l'] = bb_9_l
pitchers['k_bb_l'] = k_bb_l
pitchers['hr_9_l'] = hr_9_l
pitchers['k_pct_l'] = k_pct_l
pitchers['bb_pct_l'] = bb_pct_l
pitchers['k_bb_pct_l'] = k_bb_pct_l
pitchers['bat_avg_l'] = avg_l
pitchers['whip_l'] = whip_l
pitchers['babip_l'] = babip_l
pitchers['lob_pct_l'] = lob_pct_l
pitchers['fip_l'] = fip_l
pitchers['xfip_l'] = xfip_l
# Right handed batter stats
pitchers['k_9_r'] = k_9_r
pitchers['bb_9_r'] = bb_9_r
pitchers['k_bb_r'] = k_bb_r
pitchers['hr_9_r'] = hr_9_r
pitchers['k_pct_r'] = k_pct_r
pitchers['bb_pct_r'] = bb_pct_r
pitchers['k_bb_pct_r'] = k_bb_pct_r
pitchers['bat_avg_r'] = avg_r
pitchers['whip_r'] = whip_r
pitchers['babip_r'] = babip_r
pitchers['lob_pct_r'] = lob_pct_r
pitchers['fip_r'] = fip_r
pitchers['xfip_r'] = xfip_r
# feature engineered left handed batter stats
pitchers['k_9_l_tot'] = k_9_l_tot
pitchers['bb_9_l_tot'] = bb_9_l_tot
pitchers['k_bb_l_tot'] = k_bb_l_tot
pitchers['hr_9_l_tot'] = hr_9_l_tot
pitchers['k_pct_l_tot'] = k_pct_l_tot
pitchers['bb_pct_l_tot'] = bb_pct_l_tot
pitchers['k_bb_pct_l_tot'] = k_bb_pct_l_tot
pitchers['bat_avg_l_tot'] = avg_l_tot
pitchers['whip_l_tot'] = whip_l_tot
pitchers['babip_l_tot'] = babip_l_tot
pitchers['lob_pct_l_tot'] = lob_pct_l_tot
pitchers['fip_l_tot'] = fip_l_tot
pitchers['xfip_l_tot'] = xfip_l_tot
# feature engineered righ handed batter stats
pitchers['k_9_r_tot'] = k_9_r_tot
pitchers['bb_9_r_tot'] = bb_9_r_tot
pitchers['k_bb_r_tot'] = k_bb_r_tot
pitchers['hr_9_r_tot'] = hr_9_r_tot
pitchers['k_pct_r_tot'] = k_pct_r_tot
pitchers['bb_pct_r_tot'] = bb_pct_r_tot
pitchers['k_bb_pct_r_tot'] = k_bb_pct_r_tot
pitchers['bat_avg_r_tot'] = avg_r_tot
pitchers['whip_r_tot'] = whip_r_tot
pitchers['babip_r_tot'] = babip_r_tot
pitchers['lob_pct_r_tot'] = lob_pct_r_tot
pitchers['fip_r_tot'] = fip_r_tot
pitchers['xfip_r_tot'] = xfip_r_tot

In [23]:
pitchers

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,hr_9_r_tot,k_pct_r_tot,bb_pct_r_tot,k_bb_pct_r_tot,bat_avg_r_tot,whip_r_tot,babip_r_tot,lob_pct_r_tot,fip_r_tot,xfip_r_tot
8,STL,6/11/2021,1,670912.0,Johan Oviedo,SP,Y,NaN,3,5,...,8.181818,1.019417,0.533981,0.485437,1.204819,7.045455,1.311475,3.503185,29.686134,29.160610
17,CHC,6/11/2021,1,640464.0,Kohl Stewart,SP,Y,NaN,3,5,...,0.000000,1.000000,0.666667,0.333333,1.666667,11.250000,2.222222,3.571429,21.312896,31.131470
26,SF,6/11/2021,1,543101.0,Anthony DeSclafani,SP,N,NaN,5,3,...,2.117647,0.594203,0.260870,0.333333,0.600000,3.235294,0.692308,2.380952,11.046368,13.590725
35,WAS,6/11/2021,1,453286.0,Max Scherzer,SP,N,NaN,4,4,...,6.780269,1.420690,0.248276,1.172414,0.817844,3.928251,1.078947,3.262411,14.751341,12.336700
44,SD,6/11/2021,1,605483.0,Blake Snell,SP,Y,NaN,1,7,...,11.760000,1.997059,0.905882,1.091177,1.690476,10.733333,2.168478,5.809129,33.625634,26.171607
53,NYM,6/11/2021,1,594798.0,Jacob deGrom,SP,N,NaN,3,5,...,2.142857,2.304348,0.130435,2.173913,0.833333,3.412698,1.478261,3.865979,3.530168,5.879768
63,SEA,6/11/2021,1,643290.0,Justin Dunn,SP,N,NaN,4,5,...,4.963235,0.940860,0.510753,0.430107,0.914634,5.404412,1.000000,4.204545,22.217577,29.149787
73,CLE,6/11/2021,1,650644.0,Aaron Civale,SP,N,NaN,4,5,...,7.168142,0.990260,0.211039,0.779221,1.332180,5.973451,1.490826,3.691710,21.452179,19.931511
82,COL,6/11/2021,1,607536.0,Kyle Freeland,SP,Y,NaN,2,6,...,4.884422,0.739437,0.507042,0.232394,1.724409,8.773869,1.869767,4.924406,27.421427,30.632807
91,CIN,6/11/2021,1,641816.0,Tyler Mahle,SP,Y,NaN,4,4,...,5.268293,1.170306,0.227074,0.943231,0.980769,4.682927,1.246377,2.939068,13.963252,13.685727


**Data Description:** This worksheet provides a framework for creating dataframes for each days' players and stats. I will combine these dataframes across many days to get my final training data. These data frames have all of the stats that I could potentially be interested in, the features will be reduced for modeling as there is a significant amount of correlation between many of the features. The batters dataframe includes the batters stats against pitchers who throw with the same hand as the opposing pitcher, and includes the stats of how the opposing pitcher performs against batters with the same handedness of the batter. The pitchers dataframe contains each pitcher and their stats against both right handed and left handed batters. It also contains the number of right and left handed batters in the opposing lineup. I engineered features that are the pitchers stats vs the handedness of a batter multiplied by the number of those handed batters in the lineup. My final feature is is_nl. This is 1 if the lineup they are facing includes a pitcher and 0 if not.

**Data Dictionary:**   
team code: The team the player plays for.  
Name: The players name  
Batting order: The order the batters hit in the lineup. Is SP if player is a pitcher.  
opp_pitch_hand(batters): The hand that the opposing pitcher throws with  
opp_pitch(batters):The name of the opposing pitcher. 
opp_x(batters): This is the format for column names of stats that the opposing pitcher has against batters of the same handedness of the batter.  
(batters): There are also many columns of stats for how the batter performs against pitchers with the same handedness of the opposing pitcher.    
num_L_bats(pitchers): The number of left handed batters in the opposing lineup.  
num_R_bats(pitchers): The number of right handed batters in the opposing lineup.  
x_(l/r) (pitchers): stats describing how the pitcher performs against either left or right handed batters.    
x_(l/r)_tot (pitchers): The stats of how a pitcher performs against both hands multiplied by the number of those handed batters in the lineup.

In [24]:
scores = pd.read_csv('../datasets/6_11_target.csv', delimiter = ';')
scores.head()

,Date,GID,MLB_ID,Name,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,20210611,123x,543101,"DeSclafani, Anthony",1.0,9,1.0,43.45,6400.0,SFO,@ was,NaN,1,0
1,20210611,154q,650644,"Civale, Aaron",1.0,0,1.0,42.20,8400.0,CLE,v sea,NaN,7,0
2,20210611,121n,594798,"deGrom, Jacob",1.0,9,1.0,36.90,11300.0,NYM,v sdg,NaN,3,2
3,20210611,1495,477132,"Kershaw, Clayton",1.0,9,1.0,33.10,9800.0,LAD,v tex,NaN,12,1
4,20210611,145u,642232,"Yarbrough, Ryan",1.0,0,1.0,27.70,7700.0,TAM,v bal,NaN,4,2


In [25]:
def clean_names(name):
    name = name.replace(',','')
    name = name.replace('.','')
    name = name.replace('-hitter','')
    name = name.replace('-pitcher','')
    names = name.split()
    first = names.pop(-1)
    names.insert(0, first)
    name = ' '.join(names)
    name = name.replace('James Happ', 'JA Happ')
    return name

In [26]:
scores['Name'] = [clean_names(name) for name in scores['Name']]

In [27]:
pitch_s = scores[scores['DK posn'] == 1]
hit_s = scores[scores['DK posn'] != 1]

In [28]:
pitch_final = pd.merge(left = pitchers, right = pitch_s, how = 'inner')

In [29]:
bat_final = pd.merge(left = batters, right = hit_s,
         how = 'inner').dropna(how = 'any', subset = ['bats', 'DK pts', 'Starter'])

In [30]:
bat_final = bat_final[bat_final['dblhdr'].isna()]

In [31]:
bat_final.to_csv('../merged_data/hitters_6_11.csv')
pitch_final.to_csv('../merged_data/pitchers_6_11.csv')

In [32]:
bat_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,STL,6/11/2021,1,669242.0,Tommy Edman,1,Y,RF,R,Kohl Stewart,...,1.0,1,NaN,5.0,NaN,STL,@ chc,NaN,5,8
1,STL,6/11/2021,1,666185.0,Dylan Carlson,2,Y,CF,R,Kohl Stewart,...,1.0,2,NaN,9.0,NaN,STL,@ chc,NaN,5,8
2,STL,6/11/2021,1,502671.0,Paul Goldschmidt,3,Y,1B,R,Kohl Stewart,...,1.0,3,NaN,3.0,NaN,STL,@ chc,NaN,5,8
3,STL,6/11/2021,1,571448.0,Nolan Arenado,4,Y,3B,R,Kohl Stewart,...,1.0,4,NaN,12.0,NaN,STL,@ chc,NaN,5,8
4,STL,6/11/2021,1,641933.0,Tyler O'Neill,5,Y,LF,R,Kohl Stewart,...,1.0,5,NaN,4.0,NaN,STL,@ chc,NaN,5,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,LAD,6/11/2021,1,641355.0,Cody Bellinger,4,N,NaN,R,Mike Foltynewicz,...,1.0,4,7.0,12.0,4700.0,LAD,v tex,NaN,12,1
181,LAD,6/11/2021,1,669257.0,Will Smith,5,N,NaN,R,Mike Foltynewicz,...,1.0,5,2.0,23.0,4700.0,LAD,v tex,NaN,12,1
182,LAD,6/11/2021,1,621035.0,Chris Taylor,6,N,NaN,R,Mike Foltynewicz,...,1.0,6,47.0,10.0,4400.0,LAD,v tex,NaN,12,1
183,LAD,6/11/2021,1,666158.0,Gavin Lux,7,N,NaN,R,Mike Foltynewicz,...,1.0,7,6.0,29.0,3600.0,LAD,v tex,NaN,12,1


In [33]:
pitch_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,SF,6/11/2021,1,543101.0,Anthony DeSclafani,SP,N,NaN,5,3,...,1.0,9,1.0,43.45,6400.0,SFO,@ was,NaN,1,0
1,WAS,6/11/2021,1,453286.0,Max Scherzer,SP,N,NaN,4,4,...,1.0,9,1.0,0.75,11500.0,WAS,v sfo,NaN,0,1
2,SD,6/11/2021,1,605483.0,Blake Snell,SP,Y,NaN,1,7,...,1.0,9,1.0,6.20,9400.0,SDG,@ nym,NaN,2,3
3,NYM,6/11/2021,1,594798.0,Jacob deGrom,SP,N,NaN,3,5,...,1.0,9,1.0,36.90,11300.0,NYM,v sdg,NaN,3,2
4,SEA,6/11/2021,1,643290.0,Justin Dunn,SP,N,NaN,4,5,...,1.0,0,1.0,-4.45,7400.0,SEA,@ cle,NaN,0,7
5,CLE,6/11/2021,1,650644.0,Aaron Civale,SP,N,NaN,4,5,...,1.0,0,1.0,42.20,8400.0,CLE,v sea,NaN,7,0
6,COL,6/11/2021,1,607536.0,Kyle Freeland,SP,Y,NaN,2,6,...,1.0,9,1.0,-8.40,5500.0,COL,@ cin,NaN,5,11
7,CIN,6/11/2021,1,641816.0,Tyler Mahle,SP,Y,NaN,4,4,...,1.0,9,1.0,17.90,7900.0,CIN,v col,NaN,11,5
8,TOR,6/11/2021,1,548389.0,Ross Stripling,SP,N,NaN,4,5,...,1.0,0,1.0,12.55,6500.0,TOR,@ bos,NaN,5,6
9,BOS,6/11/2021,1,572070.0,Garrett Richards,SP,N,NaN,3,6,...,1.0,0,1.0,2.20,6700.0,BOS,v tor,NaN,6,5
